# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

## 🗒️ This notebook is divided into the following sections:
1. Load the data and process features
2. Connect to the Hopsworks feature store
3. Create feature groups and upload them to the feature store

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

First, we'll install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we'll proceed to import all the necessary libraries.

In [1]:
# Install of the packages for hopsworks
# !pip install -U hopsworks --quiet

In [2]:
# First we have to go one back in out dictionary so we can find the folder with our functions
%cd ..

# Now you can import the functions from the features folder. 
from features import electricity_prices, weater_measures # own function# This is functions we have created 

/Users/tobiasmjensen/Documents/aau_bds/m5_data-engineering-and-mlops/exam_assigment/MLOPs-Assignment-


In [3]:
# Importing of the packages for the needed libraries for the Jupyter notebook
import pandas as pd
import requests
from datetime import datetime

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')

## <span style="color:#2656a3;"> 💽 Load the historical data

The data you will use comes from three different sources:

- Electricity prices in Denmark per day from [Energinet](https://www.energidataservice.dk).
- Different meteorological observations from [Open meteo](https://www.open-meteo.com).
- Danish Calendar with the type if the date is a national holiday or not. This files is made manually by the group and is located in the "*data*" folder inside this repository.

### <span style="color:#2656a3;">💸 Electricity prices per day from Energinet
*Hvis vi skal have tariffer med i modellen, anbefales det at vi sætter en faktor på 0.2 i tidsrummet 22 - 16 og en faktor på 0.6 eller 0.7 i tidsrummet 17 - 21.*

This first dataset is Electricity prices per day from Energinet/Dataservice. Here we use 

In [4]:
# Fetching historical electricity prices data from '2022-01-01' to '2023-12-31'
electricity_df = electricity_prices.electricity_prices(
    historical=True, 
    area=["DK1"], 
    start='2022-01-01', 
    end='2023-12-31'
)

In [5]:
# Display the first 5 rows of the dataframe
electricity_df.head()

,timestamp,date,time,PriceArea,SpotPriceDKK_KWH
0,1704063600000000,2023-12-31,2023-12-31 23:00:00,DK1,0.20031
2,1704060000000000,2023-12-31,2023-12-31 22:00:00,DK1,0.21373
4,1704056400000000,2023-12-31,2023-12-31 21:00:00,DK1,0.22066
6,1704052800000000,2023-12-31,2023-12-31 20:00:00,DK1,0.26010
8,1704049200000000,2023-12-31,2023-12-31 19:00:00,DK1,0.29551


In [6]:
# Showing the information for the electricity dataframe
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 0 to 35039
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   timestamp         17520 non-null  int64         
 1   date              17520 non-null  object        
 2   time              17520 non-null  datetime64[ns]
 3   PriceArea         17520 non-null  object        
 4   SpotPriceDKK_KWH  17520 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 821.2+ KB


### <span style="color:#2656a3;">☀️💨 Forecast Renewable Energy next day from Energinet

In [7]:
# Fetching historical electricity prices data from '2022-01-01' to '2023-12-31'
forecast_renewable_energy_df = electricity_prices.forecast_renewable_energy(
    historical=True, 
    area = ["DK1"],
    start='2022-01-01', 
    end='2023-12-31'
)

In [8]:
# Display the first 5 rows of the dataframe
forecast_renewable_energy_df.head()

,timestamp,date,time,PriceArea,ForecastType,ForecastDayAhead,ForecastIntraday,ForecastCurrent
0,1704063600000000,2023-12-31,2023-12-31 23:00:00,DK1,Offshore Wind,947.750000,983.000000,1000.291687
1,1704063600000000,2023-12-31,2023-12-31 23:00:00,DK1,Onshore Wind,1210.375000,1271.458374,1501.000000
2,1704063600000000,2023-12-31,2023-12-31 23:00:00,DK1,Solar,0.000000,0.000000,0.000000
6,1704060000000000,2023-12-31,2023-12-31 22:00:00,DK1,Offshore Wind,951.250000,991.791687,1032.333374
7,1704060000000000,2023-12-31,2023-12-31 22:00:00,DK1,Onshore Wind,1298.083374,1367.416626,1502.541626


In [9]:
# Showing the information for the electricity dataframe
forecast_renewable_energy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52560 entries, 0 to 105116
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   timestamp         52560 non-null  int64         
 1   date              52560 non-null  object        
 2   time              52560 non-null  datetime64[ns]
 3   PriceArea         52560 non-null  object        
 4   ForecastType      52560 non-null  object        
 5   ForecastDayAhead  52464 non-null  float64       
 6   ForecastIntraday  34944 non-null  float64       
 7   ForecastCurrent   52560 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 3.6+ MB


### <span style="color:#2656a3;"> 🌤 Weather measurements from Open Meteo

Burde have enddate 2023-12-31. url = ("https://archive-api.open-meteo.com/v1/archive?latitude=57.048&longitude=9.9187&start_date=2022-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m")

#### <span style="color:#2656a3;"> 🕰️ Historical Weater Measures

In [10]:
# Fetching historical electricity prices data
historical_weather_df = weater_measures.historical_weater_measures(
    historical=True, 
    start = '2022-01-01', 
    end = '2023-12-31'
)

In [11]:
# Display the first 5 rows of the dataframe
historical_weather_df.head()

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1640995200000000,2022-01-01,2022-01-01 00:00:00,6.7,100,0.0,0.0,0.0,3,100,16.2,36.0
1,1640998800000000,2022-01-01,2022-01-01 01:00:00,6.6,100,0.0,0.0,0.0,3,100,16.2,30.2
2,1641002400000000,2022-01-01,2022-01-01 02:00:00,6.7,99,0.0,0.0,0.0,3,100,15.5,30.6
3,1641006000000000,2022-01-01,2022-01-01 03:00:00,6.7,100,0.0,0.0,0.0,3,100,12.7,28.8
4,1641009600000000,2022-01-01,2022-01-01 04:00:00,6.7,99,0.0,0.0,0.0,3,100,10.6,23.8


In [12]:
# Showing the information for the weather dataframe
historical_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             17520 non-null  int64         
 1   date                  17520 non-null  object        
 2   time                  17520 non-null  datetime64[ns]
 3   temperature_2m        17520 non-null  float64       
 4   relative_humidity_2m  17520 non-null  int64         
 5   precipitation         17520 non-null  float64       
 6   rain                  17520 non-null  float64       
 7   snowfall              17520 non-null  float64       
 8   weather_code          17520 non-null  int64         
 9   cloud_cover           17520 non-null  int64         
 10  wind_speed_10m        17520 non-null  float64       
 11  wind_gusts_10m        17520 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 1.6+ M

#### <span style="color:#2656a3;"> 🌈 Weater Forecast

In [15]:
# Fetching historical electricity prices data
weather_forecast_df = weater_measures.forecast_weater_measures(
    forecast_length=1
)

In [18]:
# Display the first 5 rows of the dataframe
weather_forecast_df.head()

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1713312000000000,2024-04-17,2024-04-17 00:00:00,3.8,84,0.0,0.0,0.0,2,76,5.8,9.0
1,1713315600000000,2024-04-17,2024-04-17 01:00:00,3.4,85,0.0,0.0,0.0,3,93,3.2,8.3
2,1713319200000000,2024-04-17,2024-04-17 02:00:00,2.9,88,0.0,0.0,0.0,3,97,2.5,6.5
3,1713322800000000,2024-04-17,2024-04-17 03:00:00,2.5,89,0.0,0.0,0.0,3,95,2.2,4.7
4,1713326400000000,2024-04-17,2024-04-17 04:00:00,2.2,91,0.0,0.0,0.0,3,95,1.8,3.6


In [ ]:
# Showing the information for the weather dataframe
weather_forecast_df.info()

### <span style="color:#2656a3;"> 🗓️ Calendar of Danish workdays and holidays 

In [ ]:
# Read csv file with calender
calender_df = pd.read_csv('https://raw.githubusercontent.com/Camillahannesbo/MLOPs-Assignment-/main/data/calendar_incl_holiday.csv', delimiter=';', usecols=['date', 'type'])
 
# Display the DataFrame
calender_df.head()

In [ ]:
# Formatting the date column to 'YYYY-MM-DD' dateformat
calender_df["date"] = calender_df["date"].map(lambda x: datetime.strptime(x, '%d/%m/%Y').strftime("%Y-%m-%d"))

In [ ]:
# Display the first 5 rows of the dataframe
calender_df.head()

In [ ]:
# Showing the information for the calender dataframe
calender_df.info()

## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

First we will connect to Hopsworks Feature Store so we can access and create Feature Groups.
Feature groups can also be used to define a namespace for features. For instance, in a real-life setting you would likely want to experiment with different window lengths. In that case, you can create feature groups with identical schema for each window length. 

Before you can create a feature group you need to connect to our feature store.

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

### <span style="color:#2656a3;"> 🪄 Creating Feature Groups

When creating a feature group, you must name it and designate a primary key. Additionally, it's helpful to include a description of the feature group's contents and a version number; if not defined, it will default to `1`. 

We've configured `online_enabled` as `True` to enable the feature group to be read via the Online API for a Feature View.

In [ ]:
# Creating the feature group for the weater data
weather_fg = fs.get_or_create_feature_group(
    name="weather_measurements",
    version=1,
    description="Weather measurements from Open Meteo API",
    primary_key=["date"],
    event_time="timestamp",
    online_enabled=True,
)

By now, you've only outlined metadata for the feature group. There's no data stored, nor is there a defined schema for it. To establish persistence for the feature group, you'll need to populate it with its associated data using the `insert` function

In [ ]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(historical_weather_df)

We make a descriptions for each feature we put into the feature group. In this way we are adding more information and documentation to the user

In [ ]:
weather_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the weather measurement"},
    {"name": "time", "description": "Time of the weather measurement"},
    {"name": "temperature_2m", "description": "Temperature at 2m above ground"},
    {"name": "relative_humidity_2m", "description": "Relative humidity at 2m above ground"},
    {"name": "precipitation", "description": "Precipitation"},
    {"name": "rain", "description": "Rain"},
    {"name": "snowfall", "description": "Snowfall"},   
    {"name": "weather_code", "description": "Weather code"},   
    {"name": "cloud_cover", "description": "Cloud cover"},   
    {"name": "wind_speed_10m", "description": "Wind speed at 10m above ground"},   
    {"name": "wind_gusts_10m", "description": "Wind gusts at 10m above ground"},   
]

for desc in weather_feature_descriptions: 
    weather_fg.update_feature_description(desc["name"], desc["description"])

We replicate the process for both the `electricity_fg`, `forecast_renewable_energy_fg` and `danish_holidays_fg` by establishing feature groups and inserting the dataframes into their respective feature groups.

In [ ]:
# Creating the feature group for the electricity prices
electricity_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
    description="Electricity prices from Energidata API",
    primary_key=["date"],
    online_enabled=True,
    event_time="timestamp",
)

In [ ]:
# Inserting the electricity_df into the feature group named electricity_fg
electricity_fg.insert(electricity_df)

In [ ]:
electricity_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the electricity measurement"},
    {"name": "time", "description": "Time of the electricity measurement"},
    {"name": "pricearea", "description": "Price area for the electricity measurement"},
    {"name": "spotpricedkk_kwh", "description": "Spot price in DKK per KWH"}, 
]

for desc in electricity_feature_descriptions: 
    electricity_fg.update_feature_description(desc["name"], desc["description"])

In [ ]:
# Creating the feature group for the electricity prices
forecast_renewable_energy_fg = fs.get_or_create_feature_group(
    name="forecast_renewable_energy",
    version=1,
    description="Forecast on Renewable Energy on ForecastType from Energidata API",
    primary_key=["date"],
    online_enabled=True,
    event_time="timestamp",
)

In [ ]:
# Inserting the electricity_df into the feature group named electricity_fg
forecast_renewable_energy_fg.insert(forecast_renewable_energy_df)

In [ ]:
forecast_renewable_energy_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date"},
    {"name": "time", "description": "Time for the event_time"},
    {"name": "pricearea", "description": "PriceArea (Same as bidding zone) in Denmark"},
    {"name": "forecasttype", "description": "Forecast Type such as Solar, Offshore Wind or Onshore wind"},
    {"name": "forecastdayahead", "description": "Forecast for the next day is published at 18:00 Danish time zone."},
    {"name": "forecastintraday", "description": "The forecast for the coming day at 6am Danish time zone"},
    {"name": "forecastcurrent", "description": "The forecast valid for the current time"},
]

for desc in forecast_renewable_energy_feature_descriptions: 
    forecast_renewable_energy_fg.update_feature_description(desc["name"], desc["description"])

In [ ]:
# Creating the feature group for the danish holidays
danish_holidays_fg = fs.get_or_create_feature_group(
    name="danish_holidays",
    version=1,
    description="Danish holidays calendar.",
    online_enabled=True,
    primary_key=["date"],
)

In [ ]:
# Inserting the calender_df into the feature group named danish_holidays_fg
danish_holidays_fg.insert(calender_df)

In [ ]:
danish_holidays_feature_descriptions = [
    {"name": "date", "description": "Date in the calendar"},
    {"name": "type", "description": "Holyday or not holyday"},
]

for desc in danish_holidays_feature_descriptions: 
    danish_holidays_fg.update_feature_description(desc["name"], desc["description"])

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 02: Feature Pipeline </span>

In the next notebook, you will be generating new data for the Feature Groups.